In [2]:
!pip3 install eemont

     |████████████████████████████████| 112 kB 1.8 MB/s eta 0:00:01
  Created wheel for eemont: filename=eemont-0.1.9-py3-none-any.whl size=27946 sha256=ba2331e52544fd85298226c5259c5bb7189d71ac68c32250c4ef56857adbe6e5
  Stored in directory: /home/marco/.cache/pip/wheels/d5/63/9d/b1ce24cd45ab098c70f9e40c04913cd70ed35d68149f4c8faf
Successfully built eemont


In [3]:
import ee, eemont, geemap.eefolium as geemap

In [4]:
ee.Initialize()


In [5]:
point = ee.Geometry.Point([-75.92, 2.86])


In [6]:
S2 = (ee.ImageCollection('COPERNICUS/S2_SR')
      .filterBounds(point)
      .filterDate('2020-01-01','2021-01-01'))


In [10]:
S2a = S2.maskClouds(method = 'qa')
S2b = S2.maskClouds(method = 'qa', maskCirrus = False)
S2c = S2.maskClouds()
S2d = S2.maskClouds(prob = 75)
S2e = S2.maskClouds(buffer = 500)
S2f = S2.maskClouds(dark = 0.1)
S2g = S2.maskClouds(cloudDist = 2000)
S2h = S2.maskClouds(cdi = -0.5)
S2i = S2.maskClouds(maskShadows = False)
S2j = S2.scale().maskClouds(scaledImage = True)
rgbUnscaled = {'min':0, 'max':3000, 'bands':['B4','B3','B2']}
rgbScaled = {'min':0, 'max':0.3, 'bands':['B4','B3','B2']}

In [11]:
Map = geemap.Map()
Map.centerObject(point)
Map.addLayer(S2.first(),rgbUnscaled,'No Clouds/Shadows masking')
Map.addLayer(S2a.first(),rgbUnscaled,'QA masking')
Map.addLayer(S2b.first(),rgbUnscaled,'QA masking with no cirrus masking')
Map.addLayer(S2c.first(),rgbUnscaled,'60% Cloud Probability masking')
Map.addLayer(S2d.first(),rgbUnscaled,'75% Cloud Probability masking')
Map.addLayer(S2e.first(),rgbUnscaled,'60% Cloud Probability masking with a 500 m dilation')
Map.addLayer(S2f.first(),rgbUnscaled,'60% Cloud Probability masking with a 0.1 NIR threshold')
Map.addLayer(S2g.first(),rgbUnscaled,'60% Cloud Probability masking with a 2000 m cloud shadow search range')
Map.addLayer(S2h.first(),rgbUnscaled,'60% Cloud Probability masking with a CDI of -0.5')
Map.addLayer(S2i.first(),rgbUnscaled,'60% Cloud Probability masking with no shadow masking')
Map.addLayer(S2j.first(),rgbScaled,'60% Cloud Probability masking of a scaled image')
Map.addLayerControl()
Map

In [12]:
f1 = ee.Feature(ee.Geometry.Point([3.984770,48.767221]).buffer(50),{'ID':'A'})
f2 = ee.Feature(ee.Geometry.Point([4.101367,48.748076]).buffer(50),{'ID':'B'})
fc = ee.FeatureCollection([f1,f2])

S2 = (ee.ImageCollection('COPERNICUS/S2_SR')
   .filterBounds(fc)
   .filterDate('2020-01-01','2021-01-01')
   .maskClouds()
   .scale()
   .index(['EVI','NDVI']))

# By Region
ts = S2.getTimeSeriesByRegion(reducer = [ee.Reducer.mean(),ee.Reducer.median()],
                              geometry = fc,
                              bands = ['EVI','NDVI'],
                              scale = 10)